In [7]:
from Bio import SeqIO
import subprocess
import pandas as pd
import numpy as np
import os,sys,h5py
datadir = os.path.join(os.getcwd(), '..', 'data')

In [8]:
rev_dict_alleles = {'A':0,'T':1,'U':1,'C':2,'G':3}

def parse_fasta(fasta_file):
    #read in fasta file. Return array of sequence
    fasta_sequences = SeqIO.parse(open(fasta_file),'fasta')
    sequences = []
    for fasta in fasta_sequences:
        sequences.append(str(fasta.seq))
    sequences = np.array(sequences)
    return sequences


def generate_onehot(sequence):
    #filter nonsese sequence
    filtered_list = []
    filtered_sequences = []
    for index in range(len(sequence)):
        seq = sequence[index]
        if 'N' not in seq.upper():
            filtered_sequences.append(seq)
        else:
            filtered_list.append(index)
            
    #generate one_hot matrix
    one_hot_total = []
    for seq in filtered_sequences:
        seq = seq.upper()
        seq_length = len(seq)
        one_hot = np.zeros((seq_length,4))
        for idx in range(seq_length):
            one_hot[idx,rev_dict_alleles[seq[idx]]] = 1
        
        one_hot_total.append(one_hot)
        
    one_hot_total = np.array(one_hot_total)

    return one_hot_total,filtered_list

In [9]:
#sort bed file by chromosme, split into train/test/validation set
#chr 1,3,5 for testing;
#chr 7,9 for validation;
#rest for training
count = 0

target_vec = {}
target_vec['test'] = []
target_vec['valid'] = []
target_vec['train'] = []
test_bed = open("test.bed", "w")
valid_bed = open("valid.bed", "w")
train_bed = open("train.bed", "w")

bed_f = open(datadir + '/output_sub.bed')
bed_f.readline()

for line in bed_f:
    
    chrom = line.split('\t')[0]
    class_list = line[:-1].split('\t')[3]
    class_list =class_list.split(',')
    class_list = [float(i) for i in class_list] 

    if chrom == ('chr7' or 'chr9'):
        valid_bed.write(line) 
        target_vec['valid'].append(class_list)
        
    elif chrom == ('chr1' or 'chr3' or 'chr5'):
        test_bed.write(line) 
        target_vec['test'].append(class_list)
    
    else:
        train_bed.write(line) 
        target_vec['train'].append(class_list)
        
test_bed.close()
train_bed.close()
valid_bed.close()     

In [10]:
#get fasta file from bed file. 
#change here for reference genome and input bed file
for file in ['test','train','valid']:
    cmd = 'bedtools getfasta -fi ' + datadir +'/GRCm38.fa -bed '+ file + '.bed -s -fo '+ datadir +'/'+ file+'.fa'
    subprocess.call(cmd, shell=True)
    
subprocess.call('rm ./*.bed', shell=True)

0

In [11]:
#convert fasta to onehot and store in .h5 file
output = h5py.File(datadir+'/output_sub.h5','w')
for file in ['test','train','valid']:
    fasta_path = datadir+'/'+file+'.fa'
    sequence = parse_fasta(fasta_path)
    seq_onehot,filtered_list = generate_onehot(sequence)
    seq_vec = target_vec[file]
    seq_vec = [i for j, i in enumerate(seq_vec) if j not in filtered_list]
    
    if seq_onehot.shape[0] != len(seq_vec):
        print('error')
        break
    else:
        print(len(seq_vec))
        
    output.create_dataset(file+'_seq', data=seq_onehot, dtype='float32', compression="gzip")
    output.create_dataset(file+'_label', data=seq_vec, dtype='float32', compression="gzip")
    print(file+'_seq')

38134
test_seq
436419
train_seq
26708
valid_seq


In [12]:
subprocess.call('rm ' + datadir +'/test.fa ' + datadir +'/train.fa ' + datadir +'/valid.fa', shell=True)

0